# Variational AutoEncoder (VAE)

In [ ]:
# Bisogna testare tutto passo passo
# Train Validation Test? Intendo, ci interessa la performance su dati Mai Visti?
# Plottare le qntà interessanti durante l'allenamento
# indagare #e loss e ottimizzatori

In [1]:
import torch
from torch import nn
from torch import optim

import torchvision
import torchvision.transforms as transforms

In [ ]:
# work on GPUs?
# device = torch.device("cuda:0" if torch.cuda_is_available() alse "cpu")

In [2]:
# import Dataset (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),                             
    # <-- we dont normalize here, can you guess why?
])

train_dataset = torchvision.datasets.MNIST(root='./Data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./Data', train=False, download=True, transform=transform)

100.0%
100.0%
100.0%
100.0%


In [3]:
BATCH_SIZE = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
# build model
# adesso è 1 autoencoder e basta, va reso probabilistico
class VAE(nn.Module):
    def __init__(self, input_dim, out_dim, latent_dim):
        super().__init__()
        self.flatten = nn.Flatten(1,3)
        self.layer_1 = nn.Linear(input_dim, latent_dim)
        self.activation = nn.ReLU()
        self.layer_2 = nn.Linear(latent_dim, out_dim)
        self.sigmoid = nn.Sigmoid()
        self.unflatten = nn.Unflatten(1, (1, 28, 28))

    def encode(self, x):
        x = self.flatten(x)
        x = self.layer_1(x)
        z = self.activation(x)

        return z

    def decode(self, z):
        z = self.layer_2(z)
        x_gen = self.soft(z)

        return x_gen

    def forward(self, x):
        z = encode(x)
        x_gen = decode(z)

        return x_gen

latent_dim = 2
model = VAE(28*28, 28*28, latent_dim)
# model = model.to(device)

In [6]:
# train setup
loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),
                      lr = 1e-2,
                      weight_decay=1e-6)

In [ ]:
# train loop
EPOCHS = 10
loss_train = []
loss_test = []

for epoch in range(EPOCHS):
    

In [ ]:
# evaluation and visualization